In [1]:
import requests
import json
from pprint import pprint
import os
import re
debug = False

In [2]:
class BitportBlob(object):
    '''
    The whole schmear.  All your bitport items which are also objects (BitportDirs and BitportFiles)
    '''
    def __init__(self, name):
        self.obj_list = []
        self.auth_tok = ''
        self.me_info = {}
        self.all_bpf_objects = {}
        self.dirs = []
        self.files_to_download = []
        


In [3]:
class BitportDir(object):
    '''
    One bitport directory.
    '''
    instance_names = []

    def __init__(self, name):
        self.name = name
        
        self.dictoid = {}
        self.date = ''
        self.name = ''
        self.count = 0
        self.flist = []


In [4]:
class BitportFile(object):
    '''
    One bitport file.
    '''
    instance_names = []
    fdict = {}
    def __init__(self, name):
        self.name = name
        BitportFile.instance_names.append(self.name)
        
        self.dictoid = {}
        
    #def __del__(self):
    #    print("deling", self)

In [5]:
def get_secrets_from_file(bitport_dot_file = ''):
    '''
    Keep secret stuff in a .bitport file rather than embedded in the code.
    This utility reads the file and returns a dict.  (or else creates the file.)
    One arg: bitport_dot_file is optional, and defaults to ~/.bitport
    '''
    
    
    # File should contain what dummy_file_content contains (but with real info instead of <..GOES_HERE> stuff)
    dummy_file_content = {
      "client_id":"<CLIENT_ID_GOES_HERE>",
      "client_secret": "<CLIENT_SECRET_GOES_HERE>",
      "code": "<CODE_GOES_HERE>"
    } 
    
    if bitport_dot_file == '':
        home_dir_path = os.path.expanduser('~')
        bitport_dot_file = home_dir_path + '/.bitport'
    
    if os.path.isfile(bitport_dot_file):
        if debug: print ('File {} already exists, reading...'.format(bitport_dot_file))
        

        try:
            with open(bitport_dot_file,  'r') as fh:                                                             
                mystuff = json.load(fh)  

            if debug: print("FOUND FILE CONTENTS: [[[{}]]]".format(mystuff))
        except:
            print('UHHHhhhh, problem with file ', bitport_dot_file, ' game over.')
            
            
        if re.search (r'_GOES_HERE>', mystuff['code']):
            print('Did you forget to edit the file {} ?\n Exiting!'.format(bitport_dot_file))
            exit('Game over')

        return mystuff

            
    else:
        print ('File {} not found.   Creating....'.format(bitport_dot_file))
        print ('You must go edit the file: {}'.format(bitport_dot_file))
        exit('Game over!')

        with open(bitport_dot_file, 'w') as fh:                                                             
            json.dump(dummy_file_content, fh)    
            

        return False
            

In [6]:
def get_auth_tok(url='https://api.bitport.io/v2/oauth2/access-token'):
    '''
    This method does a POST to http://bitport.io/get-access to obtain the 'code' (a.k.a. USER_CODE), 
    per the Bitport doc, "thanks to which you can get access_token" [sic]. 

    Beware that the 'code' seems to change occasionally, but the old one still works. So, it may be that
    a human needs to be logged in to a browser and hit the ./get-access site to get a new one occasionally.

    The requests.post below should yield a blob containing 3 items;
    token_type":"Bearer","expires_in":157766399,"access_token":"eyJ0eXAiOiJKV1QiLCJhbGciO...."
    
    We ony care about, (and therefore only return), the 'access_token' item.  Once we have the access_token
    we can make API calls to do useful stuff on the Bitport site.
    
    For more information see the (very terse) Bitport documentation available here: https://bitport.io/api

    '''
    
   
    
    dict1 = get_secrets_from_file()

    payload = {'type': 'application/x-www-form-urlencoded', 
               'client_id':'?', 
               'client_secret': '?', 
               'grant_type': 'code', 
               'code': '?'} 
    
    # We read those things from a file for a reason, they are needed in the payload;
    # We keep them out of the distro / git repo.   They are secrets!
    payload['client_id']     = dict1['client_id']
    payload['client_secret'] = dict1['client_secret']
    payload['code']          = dict1['code']

    if debug: print('Payload is \n'), pprint(payload)
    
    
    r = requests.post(url, data=payload)
    if ( r.status_code != 200 ):
        print("\n\nPOST was a FAILURE!!   status_code = : ", r.status_code, "\n\n")
        return False
    else:
        resp = json.loads(r.text)
        return resp['access_token']

  

In [7]:
def jsonify_req_obj(robj):
    import json
    rt = json.loads(robj.text)

    return rt

In [8]:
def bitport_api_cloud(tok):
    '''
    This returns a top level look at your Bitport cloud content.
    '''
    api_url = 'https://api.bitport.io/v2/cloud'

    header = {'Authorization': tok } 

    rr = requests.get(api_url, headers = header)
    return rr

In [9]:
def get_a_files_download_url(tok):
    
    ''' 
    Take the so-called 'fileCode', which looks like this: wzskcf8bls
    use that in the https://api.bitport.io/v2/files/<....>/stream.m3u8 url, and 
    get a result that looks like this;
    
    ['#EXTM3U',
     '#EXT-X-VERSION:4',
     '#EXT-X-TARGETDURATION:3556',
     '#EXT-X-MEDIA-SEQUENCE:0',
     '#EXTINF:3556',
     'https://s04.bitport.io/download/uGDWG96J1QRpEFQnntfzsp49vbkK1oba/1',
     '#EXT-X-ENDLIST']
    
    Then parse out and return just the https (url) piece.
    
    '''
    
    api_url = 'https://api.bitport.io/v2/files/wzskcf8bls/stream.m3u8'

    header = {'Authorization': tok } 

    rr = requests.get(api_url, headers = header)
    
    pat = r'(http.+1)\n'
    import re
    found = re.search(pat, rr.text)
    if debug: print('0=[{}]'.format(found.group(0)))
    if debug: print('1=[{}]'.format(found.group(1)))

    if debug: print('finalanswer[[[{}]]]'.format(found.group(1)[3]))

    return found.group(1)

In [10]:
def get_all_download_urls(bitport_blob_obj):
    
    print (len(bitport_blob_obj.obj_list))
    for jj in bitport_blob_obj.obj_list:

        print('\n\nworking on [{}]'.format(jj.name))
        download_url = get_a_files_download_url(bitport_blob_obj.auth_tok)
        '''
        The download_url should look something like this;
        https://s04.bitport.io/download/uGDWG96J1QRpEFQnntfzsp49vbkK1oba/1
        And undocumented Bitport feature is that there is also a .../2 url
        which provides a smaller (more compressed) MP4 version of the same
        file.   We regex the 1 into a 2 and store that as download_url_sm;
        '''

        download_url_sm = re.sub(r'/1$', '/2', download_url)
        #print("OLD URL = ", download_url)

        #print("NEW URL = ", download_url_sm)

        jj.dictoid['download_url']    = download_url
        jj.dictoid['download_url_sm'] = download_url_sm

        pprint(jj.dictoid)

In [11]:
def get_me_info_from_api(tok):
    '''
    This returns a top level look at your Bitport cloud content.
    '''
    api_url = 'https://api.bitport.io/v2/me'

    header = {'Authorization': tok } 

    rr = requests.get(api_url, headers = header)
    jj = jsonify_req_obj(rr)
    
    return jj['data']

In [12]:


def paint_top_folders(bitp_obj):
    
    for ii in range(len(bitp_obj.obj_list)):
        paint_one_row(ii, bitp_obj)
    


In [13]:
def ui(bp_obj):
    print('\n################# Info ##################\n')
    print(bp_obj.me_info)
    print('\n################# Top Folders ##################\n')

    paint_top_folders(bp_obj)
    user_input = input('Pick a number:')
    user_input = int(user_input)
    print('You picked {}\n'.format(user_input))
    if debug: print('"ui" calling "paint_one_row" with [{}] and [{}]'.format(user_input, bp_obj.obj_list))
    paint_one_row(user_input, bp_obj)

In [14]:
def bitport_api_get_folder(tok, dir_code):
    api_url = 'https://api.bitport.io/v2/cloud/' + dir_code
    header = {'Authorization': tok }
    rr = requests.get(api_url, headers = header)
    return rr

In [15]:
def parse_one_dirs_files_from_api_get_folder_results(bitport_api_get_folder_results):

    dstruct = jsonify_req_obj(bitport_api_get_folder_results)
    min_size = 10000000  # sometimes a 'sample' file is included.  We want to skip it.  Size is in bytes?
    hits = 0
    file_url_prefix = 'https://bitport.io/my-files/file/'
    good_video_file_urls = []
    #pprint(dstruct)


    #print("AAA", dstruct['data'][0])
    print("date=", dstruct['data'][0]['created_at']['date'])
    print("name=", dstruct['data'][0]['name'])
    print("count=", dstruct['data'][0]['files_count'])
    for ff in dstruct['data'][0]['files']:
        temp_dict = {}
        if re.search(r'^sample[.]', ff['name']) and ff['size'] < min_size:
            if debug: print(ff['name'], 'is a small file that starts with "sample.", skipping...')
        elif ff['screenshots'] == []:
            if debug: print(ff['name'], 'has no screenshots, skipping...')
        else:
            hits += 1
            ans = ff
            if debug: print('\tWINNER: ', ff['code'], ff['name'], ff['size'])
            temp_dict['code'] = ff['code']
            temp_dict['parent_folder_code'] = ff['parent_folder_code']
            temp_dict['size'] = ff['size']
            temp_dict['name'] = ff['name']
            temp_dict['date'] = ff['created_at']['date']
            temp_dict['url'] = file_url_prefix + ff['code']

            
            good_video_file_urls.append(temp_dict)
            
    print('parse_one_dirs_files_from_api_get_folder_results() returning with {} items'.format(hits))       
    return good_video_file_urls


In [16]:
def paint_h():
    
    
    print('{:>3} {:<12} {:<90} {:>5}'.format(
            'Row',
            'Code', 
            'Name',
            'Size(MB)')
         )
    print('{:>3} {:<12} {:<90} {:>5}'.format(
            '=' * 3,
            '=' * 12, 
            '=' * 82,
            '=' * 5)
         )

In [17]:
def paint_r(row, dictx):
    
    mb = 1024 * 1024
    
    
    if debug: print('I am "paint_r", called with [{}] and [{}]'.format(row, dictx))
    
    print('{:>3} {:<12} {:<90} {:<5.2f}'.format(
            row,
            dictx['code'], 
            dictx['name'],
            dictx['size'] / mb)
         )

In [18]:
'''def main_guy():
    print("main_guy called...")
    BiBo = BitportBlob('mmm')
    BiBo.auth_tok = get_auth_tok()
    BiBo.me_info = get_me_info_from_api(BiBo.auth_tok)
    BiBo.process_cloud_api_requests_obj(bitport_api_cloud(BiBo.auth_tok))
    # ui(BiBo)
    
    thing = bitport_api_get_folder(BiBo.auth_tok)
    print('mainguy "thing" is type:', type(thing), 'here is contents of thing:')
    pprint(jsonify_req_obj(thing))
    #dstruct = json.loads(thing)
    print('%' * 80)
    #print('idiot')
    #pprint(dstruct)
    return thing
''' 

'def main_guy():\n    print("main_guy called...")\n    BiBo = BitportBlob(\'mmm\')\n    BiBo.auth_tok = get_auth_tok()\n    BiBo.me_info = get_me_info_from_api(BiBo.auth_tok)\n    BiBo.process_cloud_api_requests_obj(bitport_api_cloud(BiBo.auth_tok))\n    # ui(BiBo)\n    \n    thing = bitport_api_get_folder(BiBo.auth_tok)\n    print(\'mainguy "thing" is type:\', type(thing), \'here is contents of thing:\')\n    pprint(jsonify_req_obj(thing))\n    #dstruct = json.loads(thing)\n    print(\'%\' * 80)\n    #print(\'idiot\')\n    #pprint(dstruct)\n    return thing\n'

In [19]:
def figure_out_dirs( the_cloud_api_requests_object):
    dict_list = []
    xx = jsonify_req_obj(the_cloud_api_requests_object)
    files_dict = xx['data'][0]['folders']
    for file_item in files_dict:
        if file_item['files_count'] > 0:

            temp_obj = BitportDir(file_item['code'])

            temp_obj.dictoid = {'name':         file_item['name'],
                                 'size':        file_item['size'], 
                                 'files_count': file_item['files_count'], 
                                 'code':    file_item['code']  
                                }

            dict_list.append(temp_obj)
    return dict_list




In [20]:
BiBo = BitportBlob('mmm')
BiBo.auth_tok = get_auth_tok()
BiBo.me_info = get_me_info_from_api(BiBo.auth_tok)
BiBo.dirs = figure_out_dirs(bitport_api_cloud(BiBo.auth_tok))


In [21]:
def print_top_level_choice(bpobj):
    paint_h()

    for zz in range(len(bpobj.dirs)):
        #print('line {}'.format(zz))
        #print(bpobj.dirs[zz].dictoid)
        paint_r(zz, bpobj.dirs[zz].dictoid)

print_top_level_choice(BiBo)

Row Code         Name                                                                                       Size(MB)
=== ============ ==================================================================================         =====
  0 8yzfuvl00s   Pinky and the Brain                                                                        17405.20
  1 leyun3xjvb   Inception.2010.CAM.XviD-TA                                                                 1451.90
  2 waq2pompk1   Game.of.Thrones.S07E02.720p.HDTV.x264-AVS[rarbg]                                           928.13
  3 2us5qpavwq   Game.of.Thrones.S07E03.1080p.WEB.h264-TBS[rarbg]                                           1701.70
  4 40p3set1sc   www.Torrenting.com  - Game.of.Thrones.S07E06.HDTV.x264.AC3-Manning                         1030.30
  5 nq17okhax4   www.Torrenting.com  - Game.of.Thrones.S07E06.HDTV.x264.AC3-Manning                         1030.30


In [22]:
for ee in BiBo.dirs:
    pprint(ee.dictoid)

{'code': '8yzfuvl00s',
 'files_count': 70,
 'name': 'Pinky and the Brain',
 'size': 18250676353}
{'code': 'leyun3xjvb',
 'files_count': 7,
 'name': 'Inception.2010.CAM.XviD-TA',
 'size': 1522427257}
{'code': 'waq2pompk1',
 'files_count': 3,
 'name': 'Game.of.Thrones.S07E02.720p.HDTV.x264-AVS[rarbg]',
 'size': 973210831}
{'code': '2us5qpavwq',
 'files_count': 3,
 'name': 'Game.of.Thrones.S07E03.1080p.WEB.h264-TBS[rarbg]',
 'size': 1784365138}
{'code': '40p3set1sc',
 'files_count': 2,
 'name': 'www.Torrenting.com  - Game.of.Thrones.S07E06.HDTV.x264.AC3-Manning',
 'size': 1080345083}
{'code': 'nq17okhax4',
 'files_count': 2,
 'name': 'www.Torrenting.com  - Game.of.Thrones.S07E06.HDTV.x264.AC3-Manning',
 'size': 1080345083}


In [23]:
#thing = bitport_api_get_folder(BiBo.auth_tok, 'leyun3xjvb')
thing = bitport_api_get_folder(BiBo.auth_tok, '8yzfuvl00s')


print(' "thing" is type:', type(thing), 'here is contents of thing:')
pprint(jsonify_req_obj(thing))
#dstruct = json.loads(thing)
print('%' * 80)
#print('idiot')
#pprint(dstruct)


 "thing" is type: <class 'requests.models.Response'> here is contents of thing:
{'data': [{'code': '8yzfuvl00s',
           'created_at': {'date': '2017-07-07 16:54:05.000000',
                          'timezone': 'UTC',
                          'timezone_type': 3},
           'files': [{'code': 'cnrdyyzwy3',
                      'conversion_status': 'converted',
                      'crc32': 1969185603,
                      'created_at': {'date': '2017-07-07 16:54:05.000000',
                                     'timezone': 'UTC',
                                     'timezone_type': 3},
                      'extension': 'avi',
                      'name': 'Pinky And The Brain - S3E46 - Dangerous '
                              'Brains.avi',
                      'parent_folder_code': '8yzfuvl00s',
                      'screenshots': {'big': 'https://static.bitport.io/6bea83b1146eb39e_l01.jpg',
                                      'medium': 'https://static.bitport.io/6bea83b1

In [25]:

a_bitport_dir_ob = BitportDir('shiiizz')
a_bitport_dir_ob.flist = parse_one_dirs_files_from_api_get_folder_results(thing)
#pprint(a_bitport_dir_ob.flist)

paint_h()
for item_num in range(len(a_bitport_dir_ob.flist)):
    paint_r(item_num, a_bitport_dir_ob.flist[item_num])
    

date= 2017-07-07 16:54:05.000000
name= Pinky and the Brain
count= 70
parse_one_dirs_files_from_api_get_folder_results() returning with 69 items
Row Code         Name                                                                                       Size(MB)
=== ============ ==================================================================================         =====
  0 cnrdyyzwy3   Pinky And The Brain - S3E46 - Dangerous Brains.avi                                         266.34
  1 e94evbqrkz   Animaniacs - S1E98 - Pinky And The Brain - Opportunity Knox.avi                            112.21
  2 c4cfacrp5h   Pinky And The Brain - S3E5 - Brain Noir.avi                                                261.12
  3 4wkov28ice   Animaniacs - S1E39 - Pinky And The Brain - Where Rodents Dare.avi                          152.03
  4 ptj13p1iqm   Pinky And The Brain - S3E9 E10 - All You Need Is Narf, Pinky's Plan.avi                    284.46
  5 pgtrzlr1y5   Pinky And The Brain - S3E34 E35 -

In [26]:
type(a_bitport_dir_ob)

__main__.BitportDir

In [27]:
a_bitport_dir_ob.date

''

In [28]:
BiBo.obj_list.index


<function list.index>

In [29]:
BiBo.dirs